In [ ]:
# @hidden_cell
# The project token is an authorization token that is used to access project resources like data sources, connections, and used by platform APIs.
from project_lib import Project
project = Project(project_id='dcf82c7f-ffb6-4b6d-a1f2-2c5e2bee0c16', project_access_token='p-53d9a8fb58b84050fb748ccd5a2a8fb9aad2fdf6')
pc = project.project_context


# Segmenting and Clustering Neighborhoods in Calgary

### Part 1: The code below will be used to scrape the Wikipedia page and generate the requested Panadas Dataframe

In [2]:
!pip install pandas
import pandas as pd

!pip install geopy
!pip install geocoder
import geocoder # import geocoder

  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491062 sha256=9a723d324831f2a5fd7c3365530b64366b131b958cae0e5cb7cb7ea5eb2ab7c7
  Stored in directory: c:\users\hz\appdata\local\pip\cache\wheels\56\b0\fe\4410d17b32f1f0c3cf54cdfb2bc04d7b4b8f4ae377e2229ba0
Successfully built future


In [4]:
# Matplotlib and associated plotting modules
!pip install matplotlib
import matplotlib.cm as cm
import matplotlib.colors as colors

!pip -q install folium
import folium # map rendering library
!pip install lxml
print('Libraries imported.')

Libraries imported.


In [5]:
!pip install geopy
from geopy.geocoders import Nominatim

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

! pip install numpy
import numpy as np

In [6]:
# Using Pands read_html() method to read the table from wiki webpage

url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_T'

df=pd.read_html(url)[1]
df

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,T1A,Medicine Hat,Central Medicine Hat,50.036460,-110.679250
1,T2A,Calgary,"Penbrooke Meadows, Marlborough",51.049680,-113.964320
2,T3A,Calgary,"Dalhousie, Edgemont, Hamptons, Hidden Valley",51.126060,-114.143158
3,T4A,Airdrie,East Airdrie,51.272450,-113.986980
4,T5A,Edmonton,"West Clareview, East Londonderry",53.5899,-113.4413
...,...,...,...,...,...
175,T5Z,Edmonton,West Lake District,53.5966,-113.4882
176,T6Z,Not assigned,Not assigned,Not assigned,Not assigned
177,T7Z,Stony Plain,Not assigned,53.5202,-114.0135
178,T8Z,Not assigned,Not assigned,Not assigned,Not assigned


In [7]:
# The code below removes cells with a borough value that is "Not assigned"

df=df[~df.Borough.str.contains("Not assigned")]
df

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,T1A,Medicine Hat,Central Medicine Hat,50.036460,-110.679250
1,T2A,Calgary,"Penbrooke Meadows, Marlborough",51.049680,-113.964320
2,T3A,Calgary,"Dalhousie, Edgemont, Hamptons, Hidden Valley",51.126060,-114.143158
3,T4A,Airdrie,East Airdrie,51.272450,-113.986980
4,T5A,Edmonton,"West Clareview, East Londonderry",53.5899,-113.4413
...,...,...,...,...,...
171,T1Z,Rocky View,Not assigned,Not assigned,Not assigned
172,T2Z,Calgary,"Douglas Glen, McKenzie Lake, Copperfield, East...",50.9023,-113.9873
173,T3Z,Redwood Meadows,Not assigned,50.9821,-114.5178
175,T5Z,Edmonton,West Lake District,53.5966,-113.4882


In [8]:
# The code below uses groupby() function to show only unique Postal Code and concatenates Neighbourhoods with the same postal code together

df=df.groupby(['Postal Code','Borough'])['Neighborhood'].apply(','.join).reset_index()
df

,Postal Code,Borough,Neighborhood
0,T1A,Medicine Hat,Central Medicine Hat
1,T1B,Medicine Hat,South Medicine Hat
2,T1C,Medicine Hat,North Medicine Hat
3,T1G,Taber,Not assigned
4,T1H,Lethbridge,North Lethbridge
...,...,...,...
139,T9N,Bonnyville,Not assigned
140,T9S,Athabasca,Not assigned
141,T9V,Lloydminster,Not assigned
142,T9W,Wainwright,Not assigned


In [9]:
# If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
# The code below will iterate through the df_dataframe and look for when neighbourhood value is "Not assigned" and replace with Borough value

df
temp=[]
for borough, neighbourhood in zip(df['Borough'],df['Neighborhood']):
    if neighbourhood=='Not assigned':
        neighbourhood=borough
    temp.append(neighbourhood)

df['Neighborhood']=temp

df

,Postal Code,Borough,Neighborhood
0,T1A,Medicine Hat,Central Medicine Hat
1,T1B,Medicine Hat,South Medicine Hat
2,T1C,Medicine Hat,North Medicine Hat
3,T1G,Taber,Taber
4,T1H,Lethbridge,North Lethbridge
...,...,...,...
139,T9N,Bonnyville,Bonnyville
140,T9S,Athabasca,Athabasca
141,T9V,Lloydminster,Lloydminster
142,T9W,Wainwright,Wainwright


In [10]:
# Since we are only interested in analyzing the boroughs that contain "Calgary" we can first clean the dataframe
# to reduce the workload required for getting location data 

# Selecting only Boroughs that contains Calgary:

calgary_df=df[df['Borough'].str.contains("Calgary")]
calgary_df

,Postal Code,Borough,Neighborhood
15,T1Y,Calgary,"Rundle, Whitehorn, Monterey Park"
17,T2A,Calgary,"Penbrooke Meadows, Marlborough"
18,T2B,Calgary,"Forest Lawn, Dover, Erin Woods"
19,T2C,Calgary,"Lynnwood Ridge, Ogden, Foothills Industrial, G..."
20,T2E,Calgary,"Bridgeland, Greenview, Zoo, YYC"
21,T2G,Calgary,"Inglewood, Burnsland, Chinatown, East Victoria..."
22,T2H,Calgary,"Highfield, Burns Industrial"
23,T2J,Calgary,"Queensland, Lake Bonavista, Willow Park, Acadia"
24,T2K,Calgary,"Thorncliffe, Tuxedo Park"
25,T2L,Calgary,"Brentwood, Collingwood, Nose Hill"


In [11]:
calgary_df=(calgary_df.set_index(['Postal Code','Borough'])
   .apply(lambda x: x.str.split(', ').explode())
   .reset_index()) 

calgary_df

,Postal Code,Borough,Neighborhood
0,T1Y,Calgary,Rundle
1,T1Y,Calgary,Whitehorn
2,T1Y,Calgary,Monterey Park
3,T2A,Calgary,Penbrooke Meadows
4,T2A,Calgary,Marlborough
...,...,...,...
109,T3M,Calgary,Mahogany
110,T3N,Calgary,Northeast Calgary
111,T3P,Calgary,Symons Valley
112,T3R,Calgary,Northwest Calgary


### The code below will be used to pull and clean the Calgary census information

In [12]:
# Using Pands read_html() method to read the table containing census information from wiki webpage

url='https://en.wikipedia.org/wiki/List_of_neighbourhoods_in_Calgary#cite_note-opendatacommunities-11'

df1=pd.read_html(url)[0]
df1=df1.drop(['Ward[12]', '2012 PopulationRank','Population(2011)[10]', '% change', 'Dwellings(2012)[10]'], axis=1)
df1=df1.rename(columns={"Name[10]": "Neighborhood", "Sector[11]": "Sector", "Type[11]":"Type",\
                      "Population(2012)[10]":"Population (2012)", "Area(km2)[11]":"Area (km^2)", "Populationdensity": "Population Density (Pop/km^2)" })

df1

,Neighborhood,Quadrant,Sector,Type,Population (2012),Area (km^2),Population Density (Pop/km^2)
0,Abbeydale,NE/SE,Northeast,Residential,5917.0,1.7,3480.6
1,Acadia,SE,South,Residential,10705.0,3.9,2744.9
2,Albert Park/Radisson Heights,SE,East,Residential,6234.0,2.5,2493.6
3,Altadore,SW,Centre,Residential,9116.0,2.9,3143.4
4,Alyth/Bonnybrook,SE,Centre,Industrial,16.0,3.8,4.2
...,...,...,...,...,...,...,...
253,Windsor Park,SW,Centre,Residential,4126.0,1.3,3173.8
254,Winston Heights/Mountview,NE,Centre,Residential,3891.0,3.0,1297
255,Woodbine,SW,South,Residential,9131.0,3.2,2853.4
256,Woodlands,SW,South,Residential,6201.0,2.8,2214.6


In [13]:
# Dropping Last row 

df1.drop(index=257,inplace=True)
df1

,Neighborhood,Quadrant,Sector,Type,Population (2012),Area (km^2),Population Density (Pop/km^2)
0,Abbeydale,NE/SE,Northeast,Residential,5917.0,1.7,3480.6
1,Acadia,SE,South,Residential,10705.0,3.9,2744.9
2,Albert Park/Radisson Heights,SE,East,Residential,6234.0,2.5,2493.6
3,Altadore,SW,Centre,Residential,9116.0,2.9,3143.4
4,Alyth/Bonnybrook,SE,Centre,Industrial,16.0,3.8,4.2
...,...,...,...,...,...,...,...
252,Willow Park,SE,South,Residential,5229.0,3.4,1537.9
253,Windsor Park,SW,Centre,Residential,4126.0,1.3,3173.8
254,Winston Heights/Mountview,NE,Centre,Residential,3891.0,3.0,1297
255,Woodbine,SW,South,Residential,9131.0,3.2,2853.4


In [14]:
# Cleaning dataset by splitting Neighborhood column on delimeter '/'

df2=(df1.drop('Neighborhood', axis=1).join(df1['Neighborhood'].str.split('/', expand=True)
                                      .stack().reset_index(level=1, drop=True).rename('Neighborhood')))
df2=df2.reset_index()
df2

,index,Quadrant,Sector,Type,Population (2012),Area (km^2),Population Density (Pop/km^2),Neighborhood
0,0,NE/SE,Northeast,Residential,5917.0,1.7,3480.6,Abbeydale
1,1,SE,South,Residential,10705.0,3.9,2744.9,Acadia
2,2,SE,East,Residential,6234.0,2.5,2493.6,Albert Park
3,2,SE,East,Residential,6234.0,2.5,2493.6,Radisson Heights
4,3,SW,Centre,Residential,9116.0,2.9,3143.4,Altadore
...,...,...,...,...,...,...,...,...
261,253,SW,Centre,Residential,4126.0,1.3,3173.8,Windsor Park
262,254,NE,Centre,Residential,3891.0,3.0,1297,Winston Heights
263,254,NE,Centre,Residential,3891.0,3.0,1297,Mountview
264,255,SW,South,Residential,9131.0,3.2,2853.4,Woodbine


### Try to scrape the above table & get location data !!!!

In [15]:
# Stripping possible spaces from data, edited 
df2.Neighborhood = df2.Neighborhood.str.strip()


Lat=[]
Lon=[]
Neighbor=[]
for i in range(0,len(df2)):
    address ='{}, Calgary City, AB'.format(df2['Neighborhood'][i])
    geolocator = Nominatim(user_agent="calgary_explorer")
    try:
        location = geolocator.geocode(address)
        Lat.append(location.latitude)
        Lon.append(location.longitude)
        Neighbor.append(df2['Neighborhood'][i])
        print(df2['Neighborhood'][i], location.latitude, location.longitude)
    except:
        print('***Unable to obtain coordinates')
        print(df2['Neighborhood'][i], 0, 0)
        Lat.append(np.nan)
        Lon.append(np.nan)
        Neighbor.append(df2['Neighborhood'][i])

        

Abbeydale 51.0588364 -113.9294129
Acadia 50.9686551 -114.0555871
***Unable to obtain coordinates
Albert Park 0 0
Radisson Heights 51.044845 -113.9901949
Altadore 51.0151043 -114.1007564
Alyth 51.0166688 -114.024294
Bonnybrook 51.0166688 -114.024294
Applewood Park 51.0446578 -113.9289305
Arbour Lake 51.1367865 -114.2023554
Aspen Woods 51.0431192 -114.2101852
Auburn Bay 50.8906047 -113.9595647
Aurora Business Park 51.1405494 -114.0627072
Banff Trail 51.0714621 -114.1116473
Bankview 51.0338869 -114.0995178
Bayview 50.9735771 -114.1125609
Beddington Heights 51.1285977 -114.0821988
Bel-Aire 50.9996931 -114.0868109
Beltline 51.0404978 -114.0725934
Bonavista Downs 50.9446559 -114.0305504
Bowness 51.0892766 -114.1930754
Braeside 50.9557843 -114.1068165
Brentwood 51.0930748 -114.1260514
Bridgeland 51.0491285 -114.0410285
Riverside 50.880115149999995 -114.01504439395605
Bridlewood 50.9007131 -114.1105152
Britannia 51.0125403 -114.0834469
Burns Industrial 51.0010615 -114.037984
Calgary Internatio

Signal Hill 51.023313 -114.1762331
Silver Springs 51.109649 -114.1954015
Silverado 50.8843663 -114.0780347
Skyline East 51.0983222 -114.0353819
Skyline West 51.0972185 -114.0476214
Skyview Ranch 51.160534 -113.9581353
Somerset 50.8987423 -114.0833678
South Airways 51.0776821 -114.0141162
South Calgary 51.0261348 -114.1019068
South Foothills 50.9720473 -113.9712264
Southview 51.0338919 -113.9946974
Southwood 50.9572479 -114.0838187
Springbank Hill 51.0289259 -114.2097933
Spruce Cliff 51.0497063 -114.1386332
St. Andrews Heights 51.0637561 -114.1246327
Starfield 50.9982367 -113.9395978
Stonegate Landing 51.1635206 -113.9850422
***Unable to obtain coordinates
Stoney 1 0 0
***Unable to obtain coordinates
Stoney 2 0 0
***Unable to obtain coordinates
Stoney 3 0 0
Stoney 4 51.1798808 -113.9844416
Strathcona Park 51.0478678 -114.1844789
Sunalta 51.04293955 -114.10141300381927
Sundance 50.8977765 -114.0427229
Sunnyside 51.0578696 -114.0787405
Sunridge 51.0750045 -113.9914787
Taradale 51.1167042 

In [16]:
df2['Latitude']=Lat
df2['Longitude']=Lon
df2['Neigh']=Neighbor


df2


,index,Quadrant,Sector,Type,Population (2012),Area (km^2),Population Density (Pop/km^2),Neighborhood,Latitude,Longitude,Neigh
0,0,NE/SE,Northeast,Residential,5917.0,1.7,3480.6,Abbeydale,51.058836,-113.929413,Abbeydale
1,1,SE,South,Residential,10705.0,3.9,2744.9,Acadia,50.968655,-114.055587,Acadia
2,2,SE,East,Residential,6234.0,2.5,2493.6,Albert Park,NaN,NaN,Albert Park
3,2,SE,East,Residential,6234.0,2.5,2493.6,Radisson Heights,51.044845,-113.990195,Radisson Heights
4,3,SW,Centre,Residential,9116.0,2.9,3143.4,Altadore,51.015104,-114.100756,Altadore
...,...,...,...,...,...,...,...,...,...,...,...
261,253,SW,Centre,Residential,4126.0,1.3,3173.8,Windsor Park,51.006165,-114.076187,Windsor Park
262,254,NE,Centre,Residential,3891.0,3.0,1297,Winston Heights,NaN,NaN,Winston Heights
263,254,NE,Centre,Residential,3891.0,3.0,1297,Mountview,51.072303,-114.047588,Mountview
264,255,SW,South,Residential,9131.0,3.2,2853.4,Woodbine,50.942554,-114.128853,Woodbine


In [18]:
# remove columns to simplify the dataframe
df3=df2.drop(['index', 'Neigh'], axis=1)

# remove all the 0 values for latitude and longitude 
df4=df3.dropna().reset_index()
df4

,index,Quadrant,Sector,Type,Population (2012),Area (km^2),Population Density (Pop/km^2),Neighborhood,Latitude,Longitude
0,0,NE/SE,Northeast,Residential,5917.0,1.7,3480.6,Abbeydale,51.058836,-113.929413
1,1,SE,South,Residential,10705.0,3.9,2744.9,Acadia,50.968655,-114.055587
2,3,SE,East,Residential,6234.0,2.5,2493.6,Radisson Heights,51.044845,-113.990195
3,4,SW,Centre,Residential,9116.0,2.9,3143.4,Altadore,51.015104,-114.100756
4,5,SE,Centre,Industrial,16.0,3.8,4.2,Alyth,51.016669,-114.024294
...,...,...,...,...,...,...,...,...,...,...
232,260,SE,South,Residential,5229.0,3.4,1537.9,Willow Park,50.960293,-114.054645
233,261,SW,Centre,Residential,4126.0,1.3,3173.8,Windsor Park,51.006165,-114.076187
234,263,NE,Centre,Residential,3891.0,3.0,1297,Mountview,51.072303,-114.047588
235,264,SW,South,Residential,9131.0,3.2,2853.4,Woodbine,50.942554,-114.128853


## Part 3: The code below will be used to explore the neighborhoods in Calgary

### First Plot the Neighborhoods obtained on a Map of Calgary 

In [19]:
# Getting the geographical coordinates of Calgary

address = 'Calgary, AB'

geolocator = Nominatim(user_agent="Cal_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Calgary City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Calgary City are 51.0534234, -114.0625892.


In [20]:
# Using Folium to the map of calgary show all the neighbourhood coordinates

calgary_map = folium.Map(location=[latitude, longitude], zoom_start=10)

# loop through the 100 crimes and add each to the map
for lat, lng, label in zip(df4.Latitude, df4.Longitude, df4.Neighborhood):
    folium.CircleMarker(
        [lat, lng],
        radius=5, # define how big you want the circle markers to be
        color='blue',
        fill=True,
        popup=label,
        fill_color='blue',
        fill_opacity=0.4
    ).add_to(calgary_map)

# show map
calgary_map


In [21]:
# Displaying Choropleth map for population density in calgary + neighbourhood markers
calgary_geo = r'D:\2.neighborhoods-master\neighborhoods-master\calgary.geojson'
#./repository/file

# create a numpy array of length 6 and has linear spacing from the minium total immigration to the maximum total immigration
threshold_scale = np.linspace(df4['Population (2012)'].min(),
                              df4['Population (2012)'].max(),
                              6, dtype=int)
threshold_scale = threshold_scale.tolist() # change the numpy array to a list
threshold_scale[-1] = threshold_scale[-1] + 1 # make sure that the last value of the list is greater than the maximum immigration

# let Folium determine the scale.
calgary_geo_map = folium.Map(location=[0, 0], zoom_start=2, tiles='Mapbox Bright')
calgary_geo_map.choropleth(
    geo_data=calgary_geo,
    data=df4,
    columns=['Neighborhood', 'Population (2012)'],
    key_on='feature.properties.name',
    threshold_scale=threshold_scale,
    fill_color='YlOrRd', 
    fill_opacity=0.7, 
    line_opacity=0.2,
    legend_name='Population 2012',
    reset=True
)
calgary_geo_map


c:\users\hz\appdata\local\programs\python\python37-32\lib\site-packages\folium\folium.py:415: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  FutureWarning


### Using Foursquare to obtain Venue data for Calgary Neighborhoods

In [60]:
# @hidden_cell
# Defining Credentials for Foursquare

CLIENT_ID = 'IEBRFVWPTAYMKDNRHEVEWUXL34RM5CYE5Y2QPPQXHXVMOD40' # your Foursquare ID
CLIENT_SECRET = 'IBDCKQLBB0LY2UIX02UQ440V5JSKWCNEC4IOUF1ZYLKJZA4B' # your Foursquare Secret
VERSION = '20201115' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
#print('CLIENT_ID: ' + CLIENT_ID)
#print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:


### The code below was used to merge df1 and calgary_df, but missing a lot of info on population in downtown area.

In [211]:
#testing to make sure neighbourhood name for df1 is correct as string

df_select=df1[df1.Neighborhood.str.contains("Whitehorn")]
print(df_select)

df_select1=calgary_df[calgary_df.Neighborhood.str.contains("Whitehorn")]
print(df_select1)

#df_select1.Neighborhood = df_select1.Neighborhood.astype(str)
#df_select.Neighborhood = df_select.Neighborhood.astype(str)

df_select1.Neighborhood = df_select1.Neighborhood.str.strip()
df_select.Neighborhood = df_select.Neighborhood.str.strip()


df_trial=df_select1.merge(df_select, on='Neighborhood')
#df_trial=pd.merge(df_select1, df_select, on='Neighborhood', how='right')
df_trial

    Neighborhood Quadrant     Sector         Type  Population (2012)  \
250    Whitehorn       NE  Northeast  Residential            11852.0   

     Area (km^2) Population Density (Pop/km^2)  
250          2.6                        4558.5  
  Postal Code  Borough Neighborhood
1         T1Y  Calgary    Whitehorn


/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,Postal Code,Borough,Neighborhood,Quadrant,Sector,Type,Population (2012),Area (km^2),Population Density (Pop/km^2)
0,T1Y,Calgary,Whitehorn,NE,Northeast,Residential,11852.0,2.6,4558.5


### The two tables will now be joined together 

In [212]:
# trying out concat to see if it works
df1.Neighborhood = df1.Neighborhood.str.strip()
calgary_df.Neighborhood = calgary_df.Neighborhood.str.strip()

df_new=calgary_df.merge(df1, on='Neighborhood', how='outer')
df_new.head(50)

,Postal Code,Borough,Neighborhood,Quadrant,Sector,Type,Population (2012),Area (km^2),Population Density (Pop/km^2)
0,T1Y,Calgary,Rundle,NE,Northeast,Residential,11258.0,2.6,4330
1,T1Y,Calgary,Whitehorn,NE,Northeast,Residential,11852.0,2.6,4558.5
2,T1Y,Calgary,Monterey Park,NE,Northeast,Residential,10470.0,3.4,3079.4
3,T2A,Calgary,Penbrooke Meadows,SE,East,Residential,8547.0,2.0,4273.5
4,T2A,Calgary,Marlborough,NE,Northeast,Residential,8434.0,2.7,3123.7
5,T2B,Calgary,Forest Lawn,SE,East,Residential,7487.0,2.3,3255.2
6,T2B,Calgary,Dover,SE,East,Residential,10474.0,4.2,2493.8
7,T2B,Calgary,Erin Woods,SE,East,Residential,6901.0,1.6,4313.1
8,T2C,Calgary,Lynnwood Ridge,NaN,NaN,NaN,NaN,NaN,NaN
9,T2C,Calgary,Ogden,SE,Southeast,Residential,8702.0,4.1,2122.4


# From here the previous code starts

In [12]:
# Using Pands read_html() method to read the table from wiki webpage

url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_T'

df=pd.read_html(url)[1]
df

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,T1A,Medicine Hat,Central Medicine Hat,50.036460,-110.679250
1,T2A,Calgary,"Penbrooke Meadows, Marlborough",51.049680,-113.964320
2,T3A,Calgary,"Dalhousie, Edgemont, Hamptons, Hidden Valley",51.126060,-114.143158
3,T4A,Airdrie,East Airdrie,51.272450,-113.986980
4,T5A,Edmonton,"West Clareview, East Londonderry",53.5899,-113.4413
...,...,...,...,...,...
175,T5Z,Edmonton,West Lake District,53.5966,-113.4882
176,T6Z,Not assigned,Not assigned,Not assigned,Not assigned
177,T7Z,Stony Plain,Not assigned,53.5202,-114.0135
178,T8Z,Not assigned,Not assigned,Not assigned,Not assigned


In [13]:
# The code below removes cells with a borough value that is "Not assigned"

df=df[~df.Borough.str.contains("Not assigned")]
df

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,T1A,Medicine Hat,Central Medicine Hat,50.036460,-110.679250
1,T2A,Calgary,"Penbrooke Meadows, Marlborough",51.049680,-113.964320
2,T3A,Calgary,"Dalhousie, Edgemont, Hamptons, Hidden Valley",51.126060,-114.143158
3,T4A,Airdrie,East Airdrie,51.272450,-113.986980
4,T5A,Edmonton,"West Clareview, East Londonderry",53.5899,-113.4413
...,...,...,...,...,...
171,T1Z,Rocky View,Not assigned,Not assigned,Not assigned
172,T2Z,Calgary,"Douglas Glen, McKenzie Lake, Copperfield, East...",50.9023,-113.9873
173,T3Z,Redwood Meadows,Not assigned,50.9821,-114.5178
175,T5Z,Edmonton,West Lake District,53.5966,-113.4882


In [14]:
# The code below uses groupby() function to show only unique Postal Code and concatenates Neighbourhoods with the same postal code together

df=df.groupby(['Postal Code','Borough'])['Neighborhood'].apply(','.join).reset_index()
df

,Postal Code,Borough,Neighborhood
0,T1A,Medicine Hat,Central Medicine Hat
1,T1B,Medicine Hat,South Medicine Hat
2,T1C,Medicine Hat,North Medicine Hat
3,T1G,Taber,Not assigned
4,T1H,Lethbridge,North Lethbridge
...,...,...,...
139,T9N,Bonnyville,Not assigned
140,T9S,Athabasca,Not assigned
141,T9V,Lloydminster,Not assigned
142,T9W,Wainwright,Not assigned


In [15]:
# If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
# The code below will iterate through the df_dataframe and look for when neighbourhood value is "Not assigned" and replace with Borough value

df
temp=[]
for borough, neighbourhood in zip(df['Borough'],df['Neighborhood']):
    if neighbourhood=='Not assigned':
        neighbourhood=borough
    temp.append(neighbourhood)

df['Neighborhood']=temp

df

,Postal Code,Borough,Neighborhood
0,T1A,Medicine Hat,Central Medicine Hat
1,T1B,Medicine Hat,South Medicine Hat
2,T1C,Medicine Hat,North Medicine Hat
3,T1G,Taber,Taber
4,T1H,Lethbridge,North Lethbridge
...,...,...,...
139,T9N,Bonnyville,Bonnyville
140,T9S,Athabasca,Athabasca
141,T9V,Lloydminster,Lloydminster
142,T9W,Wainwright,Wainwright


In [16]:
df.shape

(144, 3)

### Part 2: The code below will use geopy to generate location data for the dataframe:

In [17]:
# Since we are only interested in analyzing the boroughs that contain "Calgary" we can first clean the dataframe
# to reduce the workload required for getting location data 

# Selecting only Boroughs that contains Calgary:

calgary_df=df[df['Borough'].str.contains("Calgary")]
calgary_df

,Postal Code,Borough,Neighborhood
15,T1Y,Calgary,"Rundle, Whitehorn, Monterey Park"
17,T2A,Calgary,"Penbrooke Meadows, Marlborough"
18,T2B,Calgary,"Forest Lawn, Dover, Erin Woods"
19,T2C,Calgary,"Lynnwood Ridge, Ogden, Foothills Industrial, G..."
20,T2E,Calgary,"Bridgeland, Greenview, Zoo, YYC"
21,T2G,Calgary,"Inglewood, Burnsland, Chinatown, East Victoria..."
22,T2H,Calgary,"Highfield, Burns Industrial"
23,T2J,Calgary,"Queensland, Lake Bonavista, Willow Park, Acadia"
24,T2K,Calgary,"Thorncliffe, Tuxedo Park"
25,T2L,Calgary,"Brentwood, Collingwood, Nose Hill"


In [19]:
latitude=[]
longitude=[]
for code in calgary_df['Postal Code']:
    g = geocoder.arcgis('{}, Calgary, Alberta'.format(code))
    print(code, g.latlng)
    while (g.latlng is None):
        g = geocoder.arcgis('{}, Calgary, Alberta'.format(code))
        print(code, g.latlng)
    latlng = g.latlng
    latitude.append(latlng[0])
    longitude.append(latlng[1])

T1Y [51.08309000000003, -113.95673]
T2A [51.051690000000065, -113.95673]
T2B [51.022080000000074, -113.96029999999996]
T2C [50.97847000000007, -113.97686999999996]
T2E [51.09146000000004, -114.02752999999996]
T2G [51.02934000000005, -114.03046999999998]
T2H [50.98864000000003, -114.04734999999994]
T2J [50.94364000000007, -114.04178999999999]
T2K [51.10047000000003, -114.07255999999995]
T2L [51.107200000000034, -114.12117999999998]
T2M [51.07080000000008, -114.09382]
T2N [51.06493000000006, -114.12185999999997]
T2P [51.050410000000056, -114.07343999999995]
T2R [51.04147000000006, -114.07611999999995]
T2S [51.02094000000005, -114.07627999999994]
T2T [51.02119000000005, -114.09937999999994]
T2V [50.98514000000006, -114.09335999999996]
T2W [50.94625000000008, -114.10564999999997]
T2X [50.88561000000004, -114.08200999999997]
T2Y [50.89241000000004, -114.18064999999996]
T2Z [50.928870000000074, -113.95598999999999]
T3A [51.13254000000006, -114.13615999999996]
T3B [51.09483000000006, -114.273

In [20]:
calgary_df['Latitude']=latitude
calgary_df['Longitude']=longitude
calgary_df.tail()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


,Postal Code,Borough,Neighborhood,Latitude,Longitude
46,T3M,Calgary,"Cranston, Auburn Bay, Mahogany",50.88220,-113.93071
47,T3N,Calgary,Northeast Calgary,51.15810,-113.95653
48,T3P,Calgary,Symons Valley,51.21500,-114.14114
49,T3R,Calgary,Northwest Calgary,51.21235,-114.27693
50,T3S,Calgary,Southeast Calgary,50.94336,-113.89299


### Part 3: The code below will be used to explore and cluster the neighborhoods in Toronto

In [40]:
!pip install geopy
from geopy.geocoders import Nominatim

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

import numpy as np

In [41]:
# Getting the geographical coordinates of Calgary

address = 'Calgary, AB'

geolocator = Nominatim(user_agent="Cal_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Calgary City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Calgary City are 51.0534234, -114.0625892.


In [42]:
# @hidden_cell
# Defining Credentials for Foursquare

CLIENT_ID = 'IEBRFVWPTAYMKDNRHEVEWUXL34RM5CYE5Y2QPPQXHXVMOD40' # your Foursquare ID
CLIENT_SECRET = 'IBDCKQLBB0LY2UIX02UQ440V5JSKWCNEC4IOUF1ZYLKJZA4B' # your Foursquare Secret
VERSION = '20201115' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
#print('CLIENT_ID: ' + CLIENT_ID)
#print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:


***3.1 The code below will be used to Explore the Neighbourhoods in Calgary***

***3.1.1 First Let us Explore all the coffee shops in Calgary***

In [47]:
# User defined function for getting movie theater based on latitude and longitude
def get_nearby_coffee_shop(lat, lon, category, radius=500):
 

    url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&v={}&ll={},{}&categoryId={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, lat, lon, category, radius, LIMIT)
    
    try:
        results = requests.get(url).json()['response']['venues']
        print('*****', results)
        venues = [(item['id'],
                   item['name'],
                   get_categories(item['categories']),
                   (item['location']['lat'], item['location']['lng']),
                   format_address(item['location']),
                   item['location']['distance']) for item in results]        
    except:
        venues = []
        print(f'\nError on {url}')
    return venues

In [48]:
# Running the user defined function and pulling all relevant venues 
# Coffee Shop Id=4bf58dd8d48988d1e0931735

coffee_venues=get_nearby_coffee_shop(lat=calgary_df['Latitude'], lon=calgary_df['Longitude'], category="4bf58dd8d48988d1e0931735")


Error on https://api.foursquare.com/v2/venues/search?client_id=IEBRFVWPTAYMKDNRHEVEWUXL34RM5CYE5Y2QPPQXHXVMOD40&client_secret=IBDCKQLBB0LY2UIX02UQ440V5JSKWCNEC4IOUF1ZYLKJZA4B&v=20201115&ll=15    51.08309
17    51.05169
18    51.02208
19    50.97847
20    51.09146
21    51.02934
22    50.98864
23    50.94364
24    51.10047
25    51.10720
26    51.07080
27    51.06493
28    51.05041
29    51.04147
30    51.02094
31    51.02119
32    50.98514
33    50.94625
34    50.88561
35    50.89241
36    50.92887
37    51.13254
38    51.09483
39    51.05224
40    51.00152
41    51.13713
42    51.04606
43    51.12955
44    51.15385
45    51.13369
46    50.88220
47    51.15810
48    51.21500
49    51.21235
50    50.94336
Name: Latitude, dtype: float64,15   -113.95673
17   -113.95673
18   -113.96030
19   -113.97687
20   -114.02753
21   -114.03047
22   -114.04735
23   -114.04179
24   -114.07256
25   -114.12118
26   -114.09382
27   -114.12186
28   -114.07344
29   -114.07612
30   -114.07628
31   -114.0993

In [45]:
# User defined function to extract and process all neighbourhoods venues in Toronto

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])    
    
    # The code below contains two for loops:the first loop iterates over venues_list and gives row wise value at each iteration in terms of value_list
    # second for loop which is inside this loop iterates over values in value_list which we get from the above for loop and gives the value in the list.
    # It can be written as:        for venue_list in venue_list:
                                        # for item in venue_list:
                                            # item
            
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [46]:
# Running the user defined function and pulling all relevant venues 

calgary_venues=getNearbyVenues(names=calgary_df['Neighborhood'], latitudes=calgary_df['Latitude'], longitudes=calgary_df['Longitude'])

Rundle, Whitehorn, Monterey Park
Penbrooke Meadows, Marlborough
Forest Lawn, Dover, Erin Woods
Lynnwood Ridge, Ogden, Foothills Industrial, Great Plains
Bridgeland, Greenview, Zoo, YYC
Inglewood, Burnsland, Chinatown, East Victoria Park, Saddledome
Highfield, Burns Industrial
Queensland, Lake Bonavista, Willow Park, Acadia
Thorncliffe, Tuxedo Park
Brentwood, Collingwood, Nose Hill
Mount Pleasant, Capitol Hill, Banff Trail
Kensington, Westmont, Parkdale, University
City Centre, Calgary Tower
Connaught, West Victoria Park
Elbow Park, Britannia, Parkhill, Mission
South Calgary (Altadore / Bankview / Richmond)
Oak Ridge, Haysboro, Kingsland, Kelvin Grove, Windsor Park
Braeside, Cedarbrae, Woodbine
Midnapore, Sundance
Millrise, Somerset, Bridlewood, Evergreen
Douglas Glen, McKenzie Lake, Copperfield, East Shepard
Dalhousie, Edgemont, Hamptons, Hidden Valley
Montgomery, Bowness, Silver Springs, Greenwood
Rosscarrock, Westgate, Wildwood, Shaganappi, Sunalta
Lakeview, Glendale, Killarney, Glam

In [17]:
# Checking the size of resulting dataframe

print(calgary_venues.shape)
calgary_venues.head()

(284, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rundle, Whitehorn, Monterey Park",51.08309,-113.95673,Subway,51.084943,-113.957244,Sandwich Place
1,"Rundle, Whitehorn, Monterey Park",51.08309,-113.95673,Kevin & Amanda's No Frills,51.085108,-113.957447,Grocery Store
2,"Rundle, Whitehorn, Monterey Park",51.08309,-113.95673,Esso,51.085454,-113.957756,Gas Station
3,"Rundle, Whitehorn, Monterey Park",51.08309,-113.95673,Dollarama,51.085195,-113.956447,Discount Store
4,"Rundle, Whitehorn, Monterey Park",51.08309,-113.95673,The Black Lion,51.085214,-113.956344,Karaoke Bar


In [18]:
# Check how many venues were returned for each neighbourhood

calgary_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Braeside, Cedarbrae, Woodbine",1,1,1,1,1,1
"Brentwood, Collingwood, Nose Hill",1,1,1,1,1,1
"Bridgeland, Greenview, Zoo, YYC",3,3,3,3,3,3
"City Centre, Calgary Tower",70,70,70,70,70,70
"Connaught, West Victoria Park",89,89,89,89,89,89
"Dalhousie, Edgemont, Hamptons, Hidden Valley",1,1,1,1,1,1
"Douglas Glen, McKenzie Lake, Copperfield, East Shepard",6,6,6,6,6,6
"Forest Lawn, Dover, Erin Woods",6,6,6,6,6,6
"Hawkwood, Arbour Lake, Citadel, Ranchlands, Royal Oak, Rocky Ridge",6,6,6,6,6,6


In [19]:
# Analyzing each neighbourhood 

# one hot encoding
calgary_onehot = pd.get_dummies(calgary_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
calgary_onehot['Neighborhood'] = calgary_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [calgary_onehot.columns[-1]] + list(calgary_onehot.columns[:-1])
calgary_onehot = calgary_onehot[fixed_columns]

calgary_onehot.head()

,Neighborhood,American Restaurant,Antique Shop,Asian Restaurant,Bakery,Bank,Bar,Baseball Field,Bookstore,Boutique,...,Sports Bar,Steakhouse,Supermarket,Sushi Restaurant,Tex-Mex Restaurant,Thai Restaurant,Toy / Game Store,Vietnamese Restaurant,Warehouse Store,Yoga Studio
0,"Rundle, Whitehorn, Monterey Park",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Rundle, Whitehorn, Monterey Park",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Rundle, Whitehorn, Monterey Park",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Rundle, Whitehorn, Monterey Park",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Rundle, Whitehorn, Monterey Park",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
# Group rows by neighbourhoods and taking the frequency of occurence for each category

calgary_grouped = calgary_onehot.groupby('Neighborhood').mean().reset_index()
calgary_grouped

,Neighborhood,American Restaurant,Antique Shop,Asian Restaurant,Bakery,Bank,Bar,Baseball Field,Bookstore,Boutique,...,Sports Bar,Steakhouse,Supermarket,Sushi Restaurant,Tex-Mex Restaurant,Thai Restaurant,Toy / Game Store,Vietnamese Restaurant,Warehouse Store,Yoga Studio
0,"Braeside, Cedarbrae, Woodbine",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,"Brentwood, Collingwood, Nose Hill",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Bridgeland, Greenview, Zoo, YYC",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,"City Centre, Calgary Tower",0.000000,0.000000,0.000000,0.014286,0.014286,0.042857,0.0,0.000000,0.0,...,0.014286,0.014286,0.000000,0.057143,0.014286,0.014286,0.000000,0.014286,0.000000,0.000000
4,"Connaught, West Victoria Park",0.000000,0.000000,0.000000,0.022472,0.000000,0.044944,0.0,0.011236,0.0,...,0.000000,0.000000,0.000000,0.011236,0.000000,0.022472,0.000000,0.033708,0.000000,0.011236
5,"Dalhousie, Edgemont, Hamptons, Hidden Valley",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,"Douglas Glen, McKenzie Lake, Copperfield, East...",0.000000,0.000000,0.000000,0.000000,0.166667,0.000000,0.0,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,"Forest Lawn, Dover, Erin Woods",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,"Hawkwood, Arbour Lake, Citadel, Ranchlands, Ro...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,"Highfield, Burns Industrial",0.000000,0.000000,0.083333,0.000000,0.000000,0.000000,0.0,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.166667,0.000000


In [21]:
# Writing Function to sort each venues in descending order 

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [22]:
# Creating new dataframe to display top 10 venues in each neighbourhood

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = calgary_grouped['Neighborhood']

for ind in np.arange(calgary_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(calgary_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Braeside, Cedarbrae, Woodbine",Other Repair Shop,Yoga Studio,Gas Station,Diner,Discount Store,Donut Shop,Eastern European Restaurant,Farmers Market,Fast Food Restaurant,Food Court
1,"Brentwood, Collingwood, Nose Hill",Scenic Lookout,Convenience Store,Department Store,Diner,Discount Store,Donut Shop,Eastern European Restaurant,Farmers Market,Fast Food Restaurant,Food Court
2,"Bridgeland, Greenview, Zoo, YYC",Scenic Lookout,Restaurant,Brewery,Gas Station,Department Store,Diner,Discount Store,Donut Shop,Eastern European Restaurant,Farmers Market
3,"City Centre, Calgary Tower",Coffee Shop,Café,Sandwich Place,Hotel,Sushi Restaurant,Pub,Restaurant,Bar,Indian Restaurant,Gym
4,"Connaught, West Victoria Park",Coffee Shop,Pub,Restaurant,Bar,Café,Vietnamese Restaurant,Pizza Place,Brewery,Hotel,Lounge
5,"Dalhousie, Edgemont, Hamptons, Hidden Valley",Convenience Store,Gas Station,Department Store,Diner,Discount Store,Donut Shop,Eastern European Restaurant,Farmers Market,Fast Food Restaurant,Food Court
6,"Douglas Glen, McKenzie Lake, Copperfield, East...",Gastropub,Burger Joint,Sandwich Place,Bank,Hardware Store,Diner,Discount Store,Donut Shop,Eastern European Restaurant,Farmers Market
7,"Forest Lawn, Dover, Erin Woods",Park,Cosmetics Shop,Pub,Sporting Goods Shop,Liquor Store,Convenience Store,Frozen Yogurt Shop,Department Store,Diner,Discount Store
8,"Hawkwood, Arbour Lake, Citadel, Ranchlands, Ro...",Photography Studio,Moving Target,Lake,Grocery Store,Residential Building (Apartment / Condo),Bus Station,Yoga Studio,Frozen Yogurt Shop,Diner,Discount Store
9,"Highfield, Burns Industrial",Warehouse Store,Coffee Shop,Furniture / Home Store,Asian Restaurant,Discount Store,Pizza Place,Farmers Market,Gun Shop,Grocery Store,Fast Food Restaurant


In [23]:
# Import and run K-means to create 5 clusters


# import k-means from clustering stage
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5

calgary_grouped_clustering = calgary_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(calgary_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0,
       4, 0, 0, 0, 0], dtype=int32)

In [24]:
# Creating a new dataframe that contains the labels as well as the top 10 venues for each neighbourhood in Calgary

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

calgary_df.head()

calgary_merged = calgary_df

# merge calgary_grouped with calgary_data to add latitude/longitude for each neighborhood
calgary_merged = calgary_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

calgary_merged=calgary_merged.dropna()
calgary_merged=calgary_merged.astype({'Cluster Labels': 'int32'})



In [25]:
calgary_merged

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
15,T1Y,Calgary,"Rundle, Whitehorn, Monterey Park",51.08309,-113.95673,0,Gas Station,Discount Store,Karaoke Bar,Chinese Restaurant,Grocery Store,Sandwich Place,Furniture / Home Store,Diner,Donut Shop,Eastern European Restaurant
17,T2A,Calgary,"Penbrooke Meadows, Marlborough",51.05169,-113.95673,0,Pizza Place,Pharmacy,Pub,Fast Food Restaurant,Frozen Yogurt Shop,Cosmetics Shop,Department Store,Diner,Discount Store,Donut Shop
18,T2B,Calgary,"Forest Lawn, Dover, Erin Woods",51.02208,-113.96030,0,Park,Cosmetics Shop,Pub,Sporting Goods Shop,Liquor Store,Convenience Store,Frozen Yogurt Shop,Department Store,Diner,Discount Store
19,T2C,Calgary,"Lynnwood Ridge, Ogden, Foothills Industrial, G...",50.97847,-113.97687,0,Outdoors & Recreation,Business Service,Yoga Studio,Cosmetics Shop,Diner,Discount Store,Donut Shop,Eastern European Restaurant,Farmers Market,Fast Food Restaurant
20,T2E,Calgary,"Bridgeland, Greenview, Zoo, YYC",51.09146,-114.02753,0,Scenic Lookout,Restaurant,Brewery,Gas Station,Department Store,Diner,Discount Store,Donut Shop,Eastern European Restaurant,Farmers Market
21,T2G,Calgary,"Inglewood, Burnsland, Chinatown, East Victoria...",51.02934,-114.03047,0,Sporting Goods Shop,Comedy Club,Gym / Fitness Center,Brewery,Sports Bar,Farmers Market,Antique Shop,Café,Eastern European Restaurant,Donut Shop
22,T2H,Calgary,"Highfield, Burns Industrial",50.98864,-114.04735,0,Warehouse Store,Coffee Shop,Furniture / Home Store,Asian Restaurant,Discount Store,Pizza Place,Farmers Market,Gun Shop,Grocery Store,Fast Food Restaurant
23,T2J,Calgary,"Queensland, Lake Bonavista, Willow Park, Acadia",50.94364,-114.04179,0,Pharmacy,Grocery Store,Café,Sandwich Place,Brewery,Shopping Mall,Liquor Store,Convenience Store,Bank,Frozen Yogurt Shop
24,T2K,Calgary,"Thorncliffe, Tuxedo Park",51.10047,-114.07256,0,Liquor Store,Bank,Sandwich Place,Convenience Store,Supermarket,Pharmacy,Coffee Shop,Vietnamese Restaurant,Diner,Discount Store
25,T2L,Calgary,"Brentwood, Collingwood, Nose Hill",51.10720,-114.12118,2,Scenic Lookout,Convenience Store,Department Store,Diner,Discount Store,Donut Shop,Eastern European Restaurant,Farmers Market,Fast Food Restaurant,Food Court


In [26]:
# Visualzing the Resulting Clusters

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=8)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]

colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(calgary_merged['Latitude'], calgary_merged['Longitude'], calgary_merged['Neighborhood'], calgary_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

***After Visualizing, each cluster will also be examined to determined the discriminating venue categories that distinguish each cluster***

***Cluster 1***

In [27]:
calgary_merged.loc[calgary_merged['Cluster Labels'] == 0, calgary_merged.columns[[2] + list(range(6, calgary_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
15,"Rundle, Whitehorn, Monterey Park",Gas Station,Discount Store,Karaoke Bar,Chinese Restaurant,Grocery Store,Sandwich Place,Furniture / Home Store,Diner,Donut Shop,Eastern European Restaurant
17,"Penbrooke Meadows, Marlborough",Pizza Place,Pharmacy,Pub,Fast Food Restaurant,Frozen Yogurt Shop,Cosmetics Shop,Department Store,Diner,Discount Store,Donut Shop
18,"Forest Lawn, Dover, Erin Woods",Park,Cosmetics Shop,Pub,Sporting Goods Shop,Liquor Store,Convenience Store,Frozen Yogurt Shop,Department Store,Diner,Discount Store
19,"Lynnwood Ridge, Ogden, Foothills Industrial, G...",Outdoors & Recreation,Business Service,Yoga Studio,Cosmetics Shop,Diner,Discount Store,Donut Shop,Eastern European Restaurant,Farmers Market,Fast Food Restaurant
20,"Bridgeland, Greenview, Zoo, YYC",Scenic Lookout,Restaurant,Brewery,Gas Station,Department Store,Diner,Discount Store,Donut Shop,Eastern European Restaurant,Farmers Market
21,"Inglewood, Burnsland, Chinatown, East Victoria...",Sporting Goods Shop,Comedy Club,Gym / Fitness Center,Brewery,Sports Bar,Farmers Market,Antique Shop,Café,Eastern European Restaurant,Donut Shop
22,"Highfield, Burns Industrial",Warehouse Store,Coffee Shop,Furniture / Home Store,Asian Restaurant,Discount Store,Pizza Place,Farmers Market,Gun Shop,Grocery Store,Fast Food Restaurant
23,"Queensland, Lake Bonavista, Willow Park, Acadia",Pharmacy,Grocery Store,Café,Sandwich Place,Brewery,Shopping Mall,Liquor Store,Convenience Store,Bank,Frozen Yogurt Shop
24,"Thorncliffe, Tuxedo Park",Liquor Store,Bank,Sandwich Place,Convenience Store,Supermarket,Pharmacy,Coffee Shop,Vietnamese Restaurant,Diner,Discount Store
26,"Mount Pleasant, Capitol Hill, Banff Trail",Park,Vietnamese Restaurant,Pizza Place,Pub,Rental Car Location,Gas Station,Mediterranean Restaurant,Massage Studio,Fast Food Restaurant,Cosmetics Shop


***Cluster 2***

In [28]:
calgary_merged.loc[calgary_merged['Cluster Labels'] == 1, calgary_merged.columns[[2] + list(range(6, calgary_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
33,"Braeside, Cedarbrae, Woodbine",Other Repair Shop,Yoga Studio,Gas Station,Diner,Discount Store,Donut Shop,Eastern European Restaurant,Farmers Market,Fast Food Restaurant,Food Court


***Cluster 3***

In [29]:
calgary_merged.loc[calgary_merged['Cluster Labels'] == 2, calgary_merged.columns[[2] + list(range(6, calgary_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
25,"Brentwood, Collingwood, Nose Hill",Scenic Lookout,Convenience Store,Department Store,Diner,Discount Store,Donut Shop,Eastern European Restaurant,Farmers Market,Fast Food Restaurant,Food Court


***Cluster 4***

In [30]:
calgary_merged.loc[calgary_merged['Cluster Labels'] == 3, calgary_merged.columns[[2] + list(range(6, calgary_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
43,"Martindale, Taradale, Falconridge, Saddle Ridge",Child Care Service,Soccer Field,Yoga Studio,Gas Station,Diner,Discount Store,Donut Shop,Eastern European Restaurant,Farmers Market,Fast Food Restaurant


***Cluster 5***

In [31]:
calgary_merged.loc[calgary_merged['Cluster Labels'] == 4, calgary_merged.columns[[2] + list(range(6, calgary_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
39,"Rosscarrock, Westgate, Wildwood, Shaganappi, S...",Indian Restaurant,Gas Station,Department Store,Diner,Discount Store,Donut Shop,Eastern European Restaurant,Farmers Market,Fast Food Restaurant,Food Court
